In [13]:
from selenium import webdriver
import time 
import requests
from bs4 import BeautifulSoup
import csv 
from datetime import date
import pandas as pd

def parse(url):
    car = []
    try:
        options = webdriver.ChromeOptions()
        options.add_argument("start-maximized")
        driver = webdriver.Chrome(options=options)
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        car.append(soup.find('span', {"class":"OfferPriceCaption__price"}).text)
        lis = soup.find_all('li')
        for li in lis:
            spans = li.find_all('span', {"class":"CardInfoRow__cell"})
            if len(spans) == 2:
                car.append(spans[1].text)

    except Exception as ex:
        print(ex)

    finally:
        driver.close()
        driver.quit()
        car_cleared = [s.replace("\xa0", "") for s in car]
        return car_cleared

def main():
    columns = ['price', 'used', 'milage', 'transmission', 'colour', 'engine_capacity', 'horsepower', 'fuel_type', 'tax', 'state', 'year']
    data = pd.DataFrame(columns=columns)
    df = pd.read_csv('data_links.csv')
    for index, row in df.iterrows():
        time.sleep(2)
        info = parse(row[0])
        if len(info) >= 14:
            new_car = pd.DataFrame([[info[0].replace("₽", ""), '1', info[4].replace(" км", ""), info[10], info[6], \
                                       info[7].split('/')[0].strip().replace(" л", ""), info[7].split('/')[1].strip().replace("л.с.", ""), \
                                       info[7].split('/')[2].strip(), info[9].split('/')[0].strip().replace("₽", ""), \
                                       info[13], info[3]]], columns=data.columns)
            data = pd.concat([data, new_car], ignore_index=True)
    data_cars = pd.DataFrame(data)
    data_cars.to_csv('data_cars_BMW.csv', index=False)
    
if __name__ == '__main__':
    main()


/tmp/ipykernel_8049/842610816.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  info = parse(row[0])


'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute

In [17]:
df_cars = pd.read_csv('data_cars_BMW.csv')
def is_integer(n):
    try:
        float_n = float(n)
        int_n = int(float_n)
    except ValueError:
        return False
    else:
        return float_n == int_n
df_cars = df_cars[df_cars['tax'].apply(is_integer)]
df_cars.loc[df_cars['milage'] < 100, 'used'] = 0
df_cars.to_csv('data_cars_BMW_clean.csv', index=False)